In [3]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [4]:
#메타 파라미터 설정
learning_rate =  0.001
training_epochs = 15
batch_size =100


In [5]:
mnist_train  = dsets.MNIST(root = '/content/drive/MyDrive/Colab Notebooks/MLstudying/ML_PyTorch/',
                           train=True,
                           transform = transforms.ToTensor(),
                           download=True
                           )

mnist_test = dsets.MNIST(root='/content/drive/MyDrive/Colab Notebooks/MLstudying/ML_PyTorch/', train=False,transform=transforms.ToTensor(),download=True)

data_loader = torch.utils.data.DataLoader(
    dataset = mnist_train,
    batch_size = batch_size,
    shuffle=True,
    drop_last=True
)








In [6]:
class CNN(torch.nn.Module):

  def __init__(self):
    super(CNN, self).__init__()
    #Layer1
    #Image : (?,1,28,28)
    #Conv -> (?,32,28,28) 채널을 32개로 확장
    #Pool -> (?,32,14,14)

    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,stride=1,padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2,stride=2)
    )
    #Layer2
    #Input Data ; (?,14,14,32)
    #Conv -> (?,14,14,64)
    #Pool -> (?,7,7,64)

    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2,stride=2)
    )

    #Fully-Connected Layer, 전결합층
    #(?,7,7,64) -> (?,7x7x64) ->(?,10)
    self.fcl = torch.nn.Linear(7*7*64,10,bias=True)
    #?? batch_size,7*764인데 왜 그냥 lienar바로 때리지? 이부분 좀 헷갈리네
    #전결합층은 가중치 초기화 ?????????????????????????
    torch.nn.init.xavier_uniform_(self.fcl.weight)

  def forward(self,x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0),-1)
    out = self.fcl(out)
    return out



In [7]:
model = CNN().to(device)

criterion = torch.nn.CrossEntropyLoss().to(device) #비용함수에 soft max 함수 포함되어 있음.
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

total_batch = len(data_loader)
print('total batch :',total_batch)


total batch : 600


In [1]:
for epoch in range(training_epochs):
  avg_cost=0
  for X,Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis,Y)
    optimizer.step()

    avg_cost += cost/total_batch

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1,avg_cost))

NameError: name 'training_epochs' is not defined

In [ ]:
()